Fichero Jupyter que ejecuta el algoritmo para las 3 imagenes.



ADVERTENCIA: Para que este notebook funcione, deben estar todas las dependencias py en la misma carpeta que este notebook. Además, la imagen con la que se quiere trabajar debe estar en la misma carpeta que este notebook. Por ultimo, se debe usar Windows.

Insertar aqui el nombre de la imagen con el formato (.png, .jpg) al que se quiere aplicar el algoritmo y ejecutar dicha celda. Posteriormente, ejecute la siguiente celda.

In [6]:
nombre_imagen = "angel1.jpg"
tamano_celda = 19

Ejecutar la siguiente celda de código, una vez ejecute la superior antes:

In [7]:



import mean_shift_epanechnikov as ms
import bokeh
import superpixel
import PIL
from PIL import Image, ImageFilter
from skimage import io, color
import numpy as np
import random
import energy_generation
import foreground_estimation
import grid_utils as grid

import os
script_dir = os.path.dirname("bokeh.py")
directorio_metadatos = os.path.join(script_dir, 'metadatos/')
directorio_resultados = os.path.join(script_dir, 'resultados/')

if not os.path.isdir(directorio_resultados):
    os.makedirs(directorio_resultados)
    
if not os.path.isdir(directorio_metadatos):
    os.makedirs(directorio_metadatos)

GAUSSIAN_KERNELS = [1, 3, 5]
RGB1 = PIL.Image.open(nombre_imagen)
RGB2 = PIL.Image.open(nombre_imagen)
RGB3 = PIL.Image.open(nombre_imagen)

RGB1 = RGB1.filter(ImageFilter.GaussianBlur(GAUSSIAN_KERNELS[0]))
RGB2 = RGB2.filter(ImageFilter.GaussianBlur(GAUSSIAN_KERNELS[1]))
RGB3 = RGB3.filter(ImageFilter.GaussianBlur(GAUSSIAN_KERNELS[2]))

io.imsave(directorio_metadatos+"imagen_sigma1.png",RGB1)
io.imsave(directorio_metadatos+"imagen_sigma3.png",RGB2)
io.imsave(directorio_metadatos+"imagen_sigma5.png",RGB3)


thumbnailsigma1 = grid.thumbnail(directorio_metadatos+"imagen_sigma1.png",(200,200))
thumbnailsigma3 = grid.thumbnail(directorio_metadatos+"imagen_sigma3.png",(200,200))
thumbnailsigma5 = grid.thumbnail(directorio_metadatos+"imagen_sigma5.png",(200,200))

RGB1 = np.array(io.imread(thumbnailsigma1))
RGB2 = np.array(io.imread(thumbnailsigma3))
RGB3 = np.array(io.imread(thumbnailsigma5))

LISTA_PUNTOSRGB1 = grid.puntos_interseccion(RGB1, tamano_celda)
LISTA_PUNTOSRGB2 = grid.puntos_interseccion(RGB2, tamano_celda)
LISTA_PUNTOSRGB3 = grid.puntos_interseccion(RGB3, tamano_celda)


mean_shifterRGB1 = ms.MeanShift(kernel = 'epanechnikov_kernel')
mean_shifterRGB2 = ms.MeanShift(kernel = 'epanechnikov_kernel')
mean_shifterRGB3 = ms.MeanShift(kernel = 'epanechnikov_kernel')


mean_shift_resultRGB1 = mean_shifterRGB1.cluster(LISTA_PUNTOSRGB1,50, kernel_bandwidth = [100,255])
mean_shift_resultRGB2 = mean_shifterRGB2.cluster(LISTA_PUNTOSRGB2,50, kernel_bandwidth = [100,255])
mean_shift_resultRGB3 = mean_shifterRGB3.cluster(LISTA_PUNTOSRGB3,50, kernel_bandwidth = [100,255])


x_y_cRGB1 = foreground_estimation.cambia_formato(mean_shift_resultRGB1)
x_y_cRGB2 = foreground_estimation.cambia_formato(mean_shift_resultRGB2)
x_y_cRGB3 = foreground_estimation.cambia_formato(mean_shift_resultRGB3)

fe_estimatorRGB1 = foreground_estimation.foreground_estimation(x_y_cRGB1)
fe_estimatorRGB2 = foreground_estimation.foreground_estimation(x_y_cRGB2)
fe_estimatorRGB3 = foreground_estimation.foreground_estimation(x_y_cRGB3)



puntos_clasificadosRGB1 = fe_estimatorRGB1.classify_points()
puntos_clasificadosRGB2 = fe_estimatorRGB2.classify_points()
puntos_clasificadosRGB3 = fe_estimatorRGB3.classify_points()


egRGB1 = energy_generation.energy_generation(puntos_clasificadosRGB1,np.shape(RGB1),tamano_celda,nombre_salida=directorio_metadatos+"transformacion_distanciaRGB1.png",nombre_grid_interseccion_figura=directorio_metadatos+"grid_interseccion_figuraRGB1.png",nombre_morfologia=directorio_metadatos+"morfologiaRGB1.png")
egRGB2 = energy_generation.energy_generation(puntos_clasificadosRGB2,np.shape(RGB2),tamano_celda,nombre_salida=directorio_metadatos+"transformacion_distanciaRGB2.png",nombre_grid_interseccion_figura=directorio_metadatos+"grid_interseccion_figuraRGB2.png",nombre_morfologia=directorio_metadatos+"morfologiaRGB2.png")
egRGB3 = energy_generation.energy_generation(puntos_clasificadosRGB3,np.shape(RGB3),tamano_celda,nombre_salida=directorio_metadatos+"transformacion_distanciaRGB3.png",nombre_grid_interseccion_figura=directorio_metadatos+"grid_interseccion_figuraRGB3.png",nombre_morfologia=directorio_metadatos+"morfologiaRGB3.png")


profundidadRGB1 = egRGB1.hacer_saliency_map()
profundidadRGB2 = egRGB2.hacer_saliency_map()
profundidadRGB3 = egRGB3.hacer_saliency_map()

profundidad_unificada = energy_generation.unificar_eg(profundidadRGB1,profundidadRGB2,profundidadRGB3,directorio_metadatos)

s = superpixel.superpixel(nombre_thumbnail=thumbnailsigma1,profundidad=profundidad_unificada,nombre_salida=directorio_metadatos+"superpixel_unificado.png")
s.hacer_superpixel()

d = bokeh.bokeh(nombre_imagen,thumbnailsigma1,directorio_metadatos+"superpixel_unificado.png",directorio_metadatos,directorio_resultados)
d.difuminacion_gaussiana_fondo()

print("¡TERMINADO!")



1
Puntos en movimiento: 99
34.681132937519955
Puntos en movimiento: 96
42.38232499465835
Puntos en movimiento: 96
34.90724383759322
Puntos en movimiento: 96
35.655745475256545
Puntos en movimiento: 90
31.004237308785584
Puntos en movimiento: 86
19.050100855463498
Puntos en movimiento: 79
1.9500687348314338
Puntos en movimiento: 75
1.5745369666199858
Puntos en movimiento: 75
1.4577379733935787
Puntos en movimiento: 75
1.8465433125365822
Puntos en movimiento: 60
0.95833333332385
Puntos en movimiento: 17
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.28867513196160316
Puntos en movimiento: 5
0.2886751

C:\Users\ANGEL\Anaconda3\lib\site-packages\scipy\cluster\vq.py:148: RuntimeWarning: Some columns have standard deviation zero. The values of these columns will not change.
  RuntimeWarning)
C:\Users\ANGEL\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: metadatos/Nivel_fondo.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\ANGEL\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: metadatos/Nivel_medio_posterior.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\ANGEL\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: metadatos/Nivel_medio_medio.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\ANGEL\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: metadatos/Nivel_medio_anterior.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\ANGEL\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: metadatos/

¡TERMINADO!
